In [1]:
from copy import copy, deepcopy

In [2]:
AMPHIPODS_PER_TYPE=4

hole_coords = {coord: owner for coord, owner in zip(range(2, 9, 2), 'ABCD')}

with open("input2", "r") as f:
    f.readline()
    hallway = list(f.readline()[1:12])
    amphis = [[line[coord + 1] for coord in hole_coords]
              for line in f.readlines()
              if line[3] != "#"]

In [3]:
hallway

['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']

In [4]:
hole_coords

{2: 'A', 4: 'B', 6: 'C', 8: 'D'}

In [5]:
holes = {label: list(amphi[::-1]) for label, amphi in zip(hole_coords, zip(*amphis))}
holes

{2: ['C', 'D', 'D', 'A'],
 4: ['D', 'B', 'C', 'D'],
 6: ['B', 'A', 'B', 'C'],
 8: ['B', 'C', 'A', 'A']}

In [6]:
move_price = {a: 10**n for a, n in zip("ABCD", range(4))}
move_price

{'A': 1, 'B': 10, 'C': 100, 'D': 1000}

In [7]:
hole_cost = {length: cost
             for length, cost in zip(range(AMPHIPODS_PER_TYPE, -1, -1), range(AMPHIPODS_PER_TYPE + 1))}
hole_cost

{4: 0, 3: 1, 2: 2, 1: 3, 0: 4}

In [8]:
def to_hallway(pos, hole_coord, holes, hallway):
    if (pos in hole_coords
        or any(h != "." for h in hallway[min(hole_coord, pos): max(hole_coord, pos) + 1])
        or len(holes[hole_coord]) == 0
        or all(hole_coords[hole_coord] == resident for resident in holes[hole_coord])):
        
        return holes, hallway, 0
    
    cost = hole_cost[len(holes[hole_coord])] + abs(hole_coord - pos) + 1
    amphipod = holes[hole_coord].pop()
    cost *= move_price[amphipod]
    hallway[pos] = amphipod

    return holes, hallway, cost

def to_hole(pos, hole_coord, holes, hallway):
    if (hallway[pos] == "."
        or hallway[pos] != hole_coords[hole_coord]
        or any(hallway[pos] != resident for resident in holes[hole_coord])
        or any(h != "." for h in hallway[min(hole_coord, pos + 1): max(hole_coord, pos - 1) + 1])
        or len(holes[hole_coord]) == AMPHIPODS_PER_TYPE):
        
        return holes, hallway, 0
    
    amphipod = hallway[pos]
    cost = hole_cost[len(holes[hole_coord])] + abs(hole_coord - pos)
    cost *= move_price[amphipod]
    holes[hole_coord].append(amphipod)
    hallway[pos] = "."
    
    return holes, hallway, cost

In [9]:
def are_amphs_home(holes):
    return all(all(amph == hole_coords[coord]
                   for amph in holes[coord])
               and len(holes[coord]) == AMPHIPODS_PER_TYPE
               for coord in holes)

In [10]:
def holes_key(holes_dict):
    return tuple((k, tuple(v)) for k, v in holes_dict.items())

In [11]:
def unfinished_holes(holes):
    return {k: v for k, v in holes.items() if any(a != hole_coords[k] for a in v)}

def non_full_holes(holes):
    return {k: v for k, v in holes.items() if len(v) < AMPHIPODS_PER_TYPE}

def non_empty_pos(hallway):
    return [i for i, pos in enumerate(hallway) if pos != '.' and i not in holes]

def available_pos(hallway):
    return [i for i, pos in enumerate(hallway) if pos == '.' and i not in holes]

In [12]:
cache = {}

def move_amphipods(holes, hallway, cost):
    cache_key = (holes_key(holes), tuple(hallway), cost)
    
    if cache_key in cache:
        return cache[cache_key]
    
    if are_amphs_home(holes):
        cache[cache_key] = cost
        return cost
    
    tentative_moves = []
    
    for hole in unfinished_holes(holes):
        for pos in available_pos(hallway):
            tentative_move = to_hallway(pos, hole, deepcopy(holes), copy(hallway))
            if tentative_move[2] <= 0:
                continue
            tentative_moves.append(tentative_move)
    
    for hole in non_full_holes(holes):
        for pos in non_empty_pos(hallway):
            tentative_move = to_hole(pos, hole, deepcopy(holes), copy(hallway))
            if tentative_move[2] <= 0:
                continue
            tentative_moves.append(tentative_move)

    tentative_costs = []
    for move in tentative_moves:
        tentative_cost = cost + move_amphipods(*move)
        if tentative_cost == cost:
            continue
        tentative_costs.append(tentative_cost) 
    
    cache[cache_key] = min(tentative_costs, default=0)
    
    return cache[cache_key]

In [13]:
move_amphipods(holes, hallway, 0)

52055